In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import e3nn_jax as e3nn
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import sys
sys.path.append('.')

from freedom.tensor_products import functional
from freedom.tensor_products import gaunt_tensor_product_utils as gtp_utils

In [18]:
l1 = 1
lmax = 3

y_grid_lmax = gtp_utils.compute_y_grid(lmax=lmax, res_theta=10, res_phi=10)
y_grid_l1 = gtp_utils.compute_y_grid(lmax=l1, res_theta=10, res_phi=10)


print(jnp.allclose(
    y_grid_l1, y_grid_lmax[
        :(l1 + 1) ** 2,
        (lmax - l1): (lmax + l1) + 1, 
        (lmax - l1): (lmax + l1) + 1
    ]
))


(y_grid_l1 - y_grid_lmax[
        :(l1 + 1) ** 2,
        (lmax - l1): (lmax + l1) + 1, 
        (lmax - l1): (lmax + l1) + 1
    ])

True


Array([[[0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j]]], dtype=complex64)

In [19]:
x1 = e3nn.normal(e3nn.s2_irreps(1), jax.random.PRNGKey(0))
x2 = e3nn.normal(e3nn.s2_irreps(1), jax.random.PRNGKey(1))

In [25]:
# Test equivariance
gaunt_tensor_product_2D_fourier_fn = lambda x1, x2: functional.gaunt_tensor_product_2D_fourier(
    x1, x2,
    res_theta=100, res_phi=100, convolution_type="fft"
)
gaunt_tensor_product_2D_fourier_fn = jax.jit(gaunt_tensor_product_2D_fourier_fn)

tp_original = gaunt_tensor_product_2D_fourier_fn(
    x1, x2
)

R = e3nn.rand_matrix(jax.random.PRNGKey(3))
tp_rotated = gaunt_tensor_product_2D_fourier_fn(
    x1.transform_by_matrix(R), x2.transform_by_matrix(R)
)

tp_original.transform_by_matrix(R), tp_rotated

(1x0e+1x1o+1x2e
 [ 4.78374296 -0.12593093  2.32481884 -0.53660116 -0.18851345  2.56294331
   5.14723947  4.56951574 -0.48273899],
 1x0e+1x1o+1x2e
 [ 4.77679677 -0.1266162   2.32764077 -0.53664273 -0.18808556  2.56422721
   5.12660184  4.56959155 -0.48566406])

In [21]:
tp_s2grid = functional.gaunt_tensor_product_s2grid(
    x1, x2, 
    res_beta=100, res_alpha=99, quadrature="gausslegendre", p_val1=1, p_val2=1, s2grid_fft=False
)
tp_s2grid

1x0e+1x1o+1x2e
[ 2.69984163  0.46553642  0.18777085 -1.25098149  1.78401518 -0.0545106
 -0.45766027 -2.50128504  2.75731263]

In [24]:
y1_grid = gtp_utils.compute_y_grid(lmax, res_theta=100, res_phi=200)
z_grid = gtp_utils.compute_z_grid(lmax, res_theta=100, res_phi=200)
x1_uv = jnp.einsum("a,auv->uv", x1.array, y1_grid)
x1_restored = jnp.einsum("uv,auv->a", x1_uv.conj(), z_grid)
x1.array, x1_restored.real, jnp.isclose(x1.array, x1_restored.real, atol=5e-3)

/Users/ameyad/Documents/vector-spherical-harmonics/.venv/lib/python3.11/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=complex128 to dtype=complex64 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/Users/ameyad/Documents/vector-spherical-harmonics/.venv/lib/python3.11/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=complex128 to dtype=complex64 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/Users/ameyad/Documents/vector-spherical-harmonics/.venv/lib/python3.11/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=complex128 to dtype=complex64 with jax_numpy_dtype_promotion='standard'. In future JAX releases thi

ValueError: Size of label 'a' for operand 1 (4) does not match previous terms (16).

In [ ]:
tp_2D_fourier_direct = functional.gaunt_tensor_product_2D_fourier(
    x1, x2,
    res_theta=100, res_phi=99, convolution_type="direct"
)
tp_2D_fourier_fft = functional.gaunt_tensor_product_2D_fourier(
    x1, x2,
    res_theta=100, res_phi=99, convolution_type="fft"
)
jnp.isclose(tp_2D_fourier_direct.array, tp_2D_fourier_fft.array)

In [ ]:
# Test equivariance
tp_original = functional.gaunt_tensor_product_s2grid(
    x1, x2,
    res_beta=100, res_alpha=99, quadrature="gausslegendre", p_val1=1, p_val2=1, s2grid_fft=False

)
R = e3nn.rand_matrix(jax.random.PRNGKey(0))
tp_rotated = functional.gaunt_tensor_product_s2grid(
    x1.transform_by_matrix(R), x2.transform_by_matrix(R),
    res_beta=100, res_alpha=99, quadrature="gausslegendre", p_val1=1, p_val2=1, s2grid_fft=False
)
tp_original.transform_by_matrix(R), tp_rotated